# Side-by-side videos

Here we make side-by-side comparison videos of various methods. Videos are written to the `video/` folder.

In [ ]:
%load_ext autoreload
%autoreload 2

import json
from pathlib import Path
import mediapy
from nerfiller.experiments.base import get_experiment_folder
from nerfiller.experiments.data import blender_dataset_names, occluder_dataset_names
import numpy as np
import torch
import cv2

In [ ]:
# whether to save the individual frames and if so, the framerate
save_images = False
image_framerate = 30
use_text = False

# occluder experiments
dataset_names = occluder_dataset_names
methods = [
    ("nerfacto-nerfiller-visualize", "none", "Regions to Complete"),
    # ("nerfacto-nerfiller", "none", "Masked NeRF"),
    # ("individual-inpaint-once", "individual-lama", "LaMask"),
    # ("individual-inpaint-once", "individual-sd-image", "SD Image Cond"),
    # ("individual-inpaint-du", "none", "Inpaint + DU"),
    # ("grid-prior-du-no-depth", "none", "Ours w/o depth"),
    ("grid-prior-du", "none", "Ours"),
]
is_blender = False
num_columns = None
methods_in_row = True # if true, then concat the methods in a row. otherwise columns
name = "occluder.mp4"
color = "rgb"

# noise schedule ablation
# dataset_names = ("norway",)
# methods = [
#     ("grid-prior-du-random-noise", "none", ""),
#     ("grid-prior-du", "none", ""),
# ]
# is_blender = False
# num_columns = None
# methods_in_row = True
# name = "ablate-schedule.mp4"
# color = "rgb"

# reference
# dataset_names = ["bearears", "bearface", "jerrypillow"]
# dataset_names = ["bearears", "bearface"]
# methods = [
#     # ("nerfacto-nerfiller-visualize", "none", ""),
#     # ("nerfacto", "none", ""),
#     ("grid-prior-du-reference", "none", ""),
# ]
# is_blender = False
# num_columns = None
# methods_in_row = False
# name = "reference.mp4"
# color = "rgb"

# blender experiments
# dataset_names = blender_dataset_names
# methods = [
#     ("nerfacto", "none", "Masked NeRF"),
#     ("nerfacto", "individual-lama", "LaMask"),
#     ("nerfacto", "individual-sd-text", "SD Text Cond"),
#     ("nerfacto", "individual-sd-image", "SD Image Cond"),
#     ("nerfacto", "expanded-attention", "Extended Attention"),
#     ("nerfacto", "grid-prior-no-joint", "Grid Prior"),
#     ("nerfacto", "grid-prior", "Joint Multi-View Inpainting"),
# ]
# is_blender = True
# num_columns = None
# methods_in_row = False
# name = "blender.mp4"
# color = "rgb+depth" # hack to include training images and render next to each other

print(dataset_names)

In [ ]:
allframes = []
for dataset_name in dataset_names:
    frames = []
    for method, dataset_inpaint_method, method_name in methods:
        folder = get_experiment_folder(dataset_name, dataset_inpaint_method, method, prefix=Path("../outputs"))
        if is_blender:
            video_filename = str(Path(folder) / "final-renders/test/test.mp4")
        else:
            video_filename = str(Path(folder) / "camera_path.mp4")
        video = mediapy.read_video(video_filename)
        assert video.shape[1] == 512 and video.shape[2] == 1024
        if color == "rgb":
            video = video[:,:,:512]
        elif color == "depth":
            video = video[:,:,512:]
        elif color == "rgb+depth":
            pass
        if use_text and method_name != "":
            for i in range(len(video)):
                video[i] = cv2.putText(video[i], method_name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (28,3,252),2, cv2.LINE_AA)
        frames.append(video)
        if save_images:
            image_folder_name = "videos/" + dataset_name + "-" + dataset_inpaint_method + "-" + method
            Path(image_folder_name).mkdir(parents=True, exist_ok=True)
            indices = torch.arange(0, len(video)-1, image_framerate)
            for i in indices:
                image_filename = image_folder_name + f"/frame_{i:06d}.png"
                mediapy.write_image(image_filename, video[i])
    if methods_in_row:
        if num_columns:
            framesrows = []
            assert len(frames) % num_columns == 0, "frames must be divisible by num_columns"
            for i in range(len(frames)//num_columns):
                framesrow = np.concatenate(frames[i*num_columns:i*num_columns+num_columns], axis=2)
                framesrows.append(framesrow)
            frames = np.concatenate(framesrows, axis=1)
        else:
            frames = np.concatenate(frames, axis=2)
    else:
        frames = np.concatenate(frames, axis=1)
    fps = video.metadata.fps
    filename = "videos/" + dataset_name + ".mp4"
    print(filename)
    mediapy.write_video(filename, frames, fps=fps)
    allframes.append(frames)
    print(len(frames), fps)
    # TODO: assert that len of frames and fps are all the same
# uncomment to save all
# if methods_in_row:
#     allframes = np.concatenate(allframes, axis=1)
# else:
#     allframes = np.concatenate(allframes, axis=2)
# filename = "videos/" + name
# print(filename)
# mediapy.write_video(filename, allframes, fps=fps)

In [ ]:
# video.shape